# **Phase 0: Setting Up**

In [1]:
import sys
print(sys.executable)

/Users/ivantan/Desktop/rag-demo/.venv/bin/python


In [2]:
import os
from pathlib import Path
from typing import List, Dict, Any, Optional
import logging
from datetime import datetime
from dotenv import load_dotenv # Environment and configuration

# LangChain core
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage

# Document loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    Docx2txtLoader,
)

from langchain_community.vectorstores import Chroma # Vector store
from langchain_google_genai import ChatGoogleGenerativeAI # LLM
from rank_bm25 import BM25Okapi # BM25 for hybrid search
import tiktoken # Token counting

print("Core libraries imported successfully!")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Core libraries imported successfully!
Timestamp: 2026-02-08 23:49:52


In [3]:
# This helps track the RAG pipeline execution and identify bottlenecks
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger('IndustrialRAG')

# Test logging
logger.info("Logging system initialized")
logger.info("Starting Industrial RAG System Development")
print("Logging configured successfully!")

2026-02-08 23:49:52 - IndustrialRAG - INFO - Logging system initialized
2026-02-08 23:49:52 - IndustrialRAG - INFO - Starting Industrial RAG System Development


Logging configured successfully!


In [4]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if not GOOGLE_API_KEY:
    raise ValueError("GOOGLE_API_KEY not found in .env file!")

# Mask the key for security when logging
masked_key = f"{GOOGLE_API_KEY[:8]}...{GOOGLE_API_KEY[-4:]}"
logger.info(f"API Key loaded: {masked_key}")

print(f"Google API Key loaded successfully!")
print(f"Masked Key: {masked_key}")

2026-02-08 23:49:52 - IndustrialRAG - INFO - API Key loaded: AIzaSyAU...LT-U


Google API Key loaded successfully!
Masked Key: AIzaSyAU...LT-U


In [5]:
# Uses tiktoken (OpenAI's tokenizer) as a proxy for token estimation
def count_tokens(text: str, encoding_name: str = "cl100k_base") -> int:
    """
    Count the number of tokens in a text string.
    """
    try:
        encoding = tiktoken.get_encoding(encoding_name)
        tokens = encoding.encode(text)
        return len(tokens)
    except Exception as e:
        logger.error(f"Error counting tokens: {e}")
        # Fallback: rough estimation (1 token ≈ 4 characters)
        return len(text) // 4

def analyze_document(text: str) -> Dict[str, Any]:
    """   
    Strategy:
    - Short docs (<2000 tokens): chunk_size=500, overlap=100
    - Medium docs (2000-10000 tokens): chunk_size=1000, overlap=200
    - Long docs (>10000 tokens): chunk_size=1500, overlap=300
    Returns: Dictionary with token count and suggested chunk parameters
    """
    token_count = count_tokens(text)
    word_count = len(text.split())
    char_count = len(text)
    
    # Determine optimal chunk size based on document length
    if token_count < 2000:
        chunk_size = 500
        chunk_overlap = 100
        strategy = "small"
    elif token_count < 10000:
        chunk_size = 1000
        chunk_overlap = 200
        strategy = "medium"
    else:
        chunk_size = 1500
        chunk_overlap = 300
        strategy = "large"
    
    analysis = {
        "token_count": token_count,
        "word_count": word_count,
        "char_count": char_count,
        "suggested_chunk_size": chunk_size,
        "suggested_overlap": chunk_overlap,
        "strategy": strategy
    }
    
    logger.info(f"Document analysis: {token_count} tokens, strategy={strategy}")
    
    return analysis

In [6]:
# Test the token counter
test_text = """
Artificial intelligence (AI) is transforming the world at an unprecedented pace.
Machine learning, a subset of AI, enables computers to learn from data without
explicit programming. Deep learning, using neural networks, has achieved remarkable
results in computer vision, natural language processing, and speech recognition.
"""

analysis = analyze_document(test_text)

print("Token counter utility created!")
print(f"\nTest Document Analysis:")
print(f"    Tokens: {analysis['token_count']}")
print(f"    Words: {analysis['word_count']}")
print(f"    Characters: {analysis['char_count']}")
print(f"    Suggested chunk size: {analysis['suggested_chunk_size']}")
print(f"    Suggested overlap: {analysis['suggested_overlap']}")
print(f"    Strategy: {analysis['strategy']}")

2026-02-08 23:49:52 - IndustrialRAG - INFO - Document analysis: 59 tokens, strategy=small


Token counter utility created!

Test Document Analysis:
    Tokens: 59
    Words: 44
    Characters: 326
    Suggested chunk size: 500
    Suggested overlap: 100
    Strategy: small


# **Phase 2: Document Processing Pipeline.**

In [7]:
from langchain_community.document_loaders import UnstructuredFileLoader, DirectoryLoader, WebBaseLoader

def load(source: str) -> List[Document]:
    """Auto-detect and load file or directory using LangChain loaders."""
    
    if source.startswith(("http://", "https://")):
        loader = WebBaseLoader(source)
        return loader.load()
    
    source_path = Path(source)
    
    if source_path.is_file():
        loader = UnstructuredFileLoader(str(source_path))
        return loader.load()
        
    elif source_path.is_dir():
        loader = DirectoryLoader(
            str(source_path),
            glob="**/*",
            loader_cls=UnstructuredFileLoader,
            use_multithreading=True
        )
        return loader.load()
    
    else:
        raise ValueError(f"Invalid source: {source}")

2026-02-08 23:49:52 - langchain_community.utils.user_agent - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
dir_doc = load("test_documents")
web_doc = load("https://karpathy.ai/")
all_doc = dir_doc + web_doc

2026-02-08 23:49:54 - unstructured - WARNING - libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
2026-02-08 23:49:55 - pikepdf._core - INFO - pikepdf C++ to Python logger bridge initialized


# **Phase 3: Chunking and Embeddings**

In [9]:
# Phase 3: Chunking - Using LangChain's RecursiveCharacterTextSplitter

from langchain_text_splitters import RecursiveCharacterTextSplitter

def create_chunks(documents: List[Document], chunk_size: int = 1000, chunk_overlap: int = 200) -> List[Document]:
    """
    Split documents into chunks using RecursiveCharacterTextSplitter.
    Tries to split on paragraphs, then sentences, maintaining semantic coherence.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", ";", " ", ""],
        is_separator_regex=False,
    )
    
    chunks = text_splitter.split_documents(documents)
    logger.info(f"Split {len(documents)} documents into {len(chunks)} chunks")
    
    return chunks

In [10]:
# Chunk all documents
all_chunks = create_chunks(all_doc)
print(f"Created {len(all_chunks)} chunks from {len(all_doc)} documents\n")
print("\nFirst 100 chars in the first doc:\n", all_chunks[0].page_content[:100])
print("\nFirst 100 chars in the second doc:\n", all_chunks[1].page_content[:100])

2026-02-08 23:49:56 - IndustrialRAG - INFO - Split 4 documents into 113 chunks


Created 113 chunks from 4 documents


First 100 chars in the first doc:
 # Retrieval

Augmented Generation (RAG) Systems

## Introduction Retrieval-Augmented Generation (RAG

First 100 chars in the second doc:
 5. **Query Processing**: When a user asks a question, it's converted into an embedding using the sam


In [11]:
# Phase 3: Embeddings - Using HuggingFace sentence-transformers

from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'mps'},  # Use Apple Silicon GPU
    encode_kwargs={'normalize_embeddings': True}
)

logger.info("Embeddings model initialized: all-MiniLM-L6-v2")
print("Embeddings model ready (384 dimensions)")


2026-02-08 23:49:56 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2026-02-08 23:49:56 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 23:49:56 - huggingface_hub.utils._http - WARNING - Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.
2026-02-08 23:49:56 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/modules.json "HTTP/1.1 200 OK"
2026-02-08 23:49:57 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config_sentence_transformers.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 23:49:57 - httpx - INFO - HTTP Request: HEAD htt

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
2026-02-08 23:49:58 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 23:49:58 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207416be6d2e6f8de32d1f16199bf/config.json "HTTP/1.1 200 OK"
2026-02-08 23:49:59 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-08 23:49:59 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/s

Embeddings model ready (384 dimensions)


In [12]:
# Vector Store - Index all chunks in ChromaDB

from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents=all_chunks,
    embedding=embeddings,
    collection_name="rag_knowledge_base"
)

logger.info(f"Vector store created with {len(all_chunks)} chunks")
print(f"Vector store indexed: {len(all_chunks)} chunks ready for retrieval")

2026-02-08 23:50:01 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2026-02-08 23:50:02 - IndustrialRAG - INFO - Vector store created with 113 chunks


Vector store indexed: 113 chunks ready for retrieval


# **Phase 4: Retrieval**

In [13]:
# Phase 4: Retrieval - Create retriever from vector store

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}  # Retrieve top 10 candidates for re-ranking
)

logger.info("Retriever created with k=10")
print("Retriever ready: will fetch top 10 candidates")

2026-02-08 23:50:02 - IndustrialRAG - INFO - Retriever created with k=10


Retriever ready: will fetch top 10 candidates


## Retrieval Test

In [36]:
def top_k_snapshot(query: str, results: List[Document], top_k: int = 5) -> None:
    print(f"Query: '{query}'")
    print(f"Retrieved {len(results)} documents\n")
    
    for i, doc in enumerate(results[:top_k], 1):
        print(f"Result [{i}]:")
        print(f"{doc.page_content[:150].replace('\n', ' ')}...")
        print(f"Source: {doc.metadata.get('source', 'N/A')}\n")

retrieval_results = retriever.invoke(test_query) # This is where the actual retrieval happens - executes the search and returns the actual documents
top_k_snapshot(test_query, retrieval_results, 5)

Query: 'What are the benefits of RAG systems?'
Retrieved 10 documents

Result [1]:
9  Broader Impact  This work offers several positive societal beneﬁts over previous work: the fact that it is more strongly grounded in real factual k...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [2]:
. Like T5 [51] or BART, RAG can be ﬁne-tuned on any seq2seq task, whereby both the generator and retriever are jointly learned....
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [3]:
Acknowledgments  The authors would like to thank the reviewers for their thoughtful and constructive feedback on this paper, as well as HuggingFace fo...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [4]:
Effect of Retrieving more documents Models are trained with either 5 or 10 retrieved latent documents, and we do not observe signiﬁcant differences in...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [5]:
4.3  Jeopard

## Rerank

In [31]:
# Re-ranking with Cross-Encoder (production approach)

from sentence_transformers import CrossEncoder

# Initialize cross-encoder model for re-ranking
# This model is specifically trained to score query-document pairs
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def rerank_documents(query: str, documents: List[Document], top_k: int = 3) -> List[Document]:
    """
    Re-rank documents using cross-encoder.
    
    Args:
        query: User query
        documents: List of retrieved documents
        top_k: Number of top documents to return after re-ranking
    
    Returns:
        Top-k re-ranked documents
    """
    # Create query-document pairs for scoring
    pairs = [[query, doc.page_content] for doc in documents]
    
    # Get relevance scores from cross-encoder
    scores = cross_encoder.predict(pairs)
    
    # Sort documents by score (descending)
    scored_docs = list(zip(documents, scores))
    scored_docs.sort(key=lambda x: x[1], reverse=True)
    
    # Return top-k documents
    top_docs = [doc for doc, score in scored_docs[:top_k]]
    
    logger.info(f"Re-ranked {len(documents)} docs to top {top_k}")
    
    return top_docs

print("Cross-encoder re-ranker initialized")
print("Model: cross-encoder/ms-marco-MiniLM-L-6-v2")

2026-02-09 08:34:33 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:33 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:33 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/cross-encoder/ms-marco-MiniLM-L6-v2/c5ee24cb16019beea0893ab7796b1df96625c6b8/config.json "HTTP/1.1 200 OK"
2026-02-09 08:34:33 - httpx - INFO - HTTP Request: GET https://huggingface.co/api/resolve-cache/models/cross-encoder/ms-marco-MiniLM-L6-v2/c5ee24cb16019beea0893ab7796b1df96625c6b8/config.json "HTTP/1.1 200 OK"


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

2026-02-09 08:34:34 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2/resolve/main/model.safetensors "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:34 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2/resolve/main/model.safetensors "HTTP/1.1 302 Found"
2026-02-09 08:34:34 - httpx - INFO - HTTP Request: GET https://huggingface.co/api/models/cross-encoder/ms-marco-MiniLM-L6-v2/xet-read-token/c5ee24cb16019beea0893ab7796b1df96625c6b8 "HTTP/1.1 200 OK"


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/105 [00:00<?, ?it/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

2026-02-09 08:34:42 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2/resolve/main/chat_template.jinja "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:42 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2/resolve/main/chat_template.jinja "HTTP/1.1 404 Not Found"
2026-02-09 08:34:42 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:42 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/cross-encoder/ms-marco-MiniLM-L6-v2/resolve/main/README.md "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:42 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/cross-encoder/ms-marco-MiniLM-L6-v2/c5ee24cb16019beea0893ab7796b1df96625c6b8/README.md "HTTP/1.1 200 OK"
2026-02-09 08:34:42 - httpx - INFO - HTTP Request: GET https://huggingface.co/api/resolve-

README.md: 0.00B [00:00, ?B/s]

2026-02-09 08:34:42 - sentence_transformers.cross_encoder.CrossEncoder - INFO - Use pytorch device: mps
2026-02-09 08:34:43 - httpx - INFO - HTTP Request: GET https://huggingface.co/api/models/cross-encoder/ms-marco-MiniLM-L-6-v2 "HTTP/1.1 307 Temporary Redirect"
2026-02-09 08:34:43 - httpx - INFO - HTTP Request: GET https://huggingface.co/api/models/cross-encoder/ms-marco-MiniLM-L6-v2 "HTTP/1.1 200 OK"


Cross-encoder re-ranker initialized
Model: cross-encoder/ms-marco-MiniLM-L-6-v2


In [37]:
rerank_results = rerank_documents(test_query, retrieval_results)
top_k_snapshot(test_query, rerank_results, 3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-09 08:44:25 - IndustrialRAG - INFO - Re-ranked 10 docs to top 3


Query: 'What are the benefits of RAG systems?'
Retrieved 3 documents

Result [1]:
With these advantages also come potential downsides: Wikipedia, or any potential external knowledge source, will probably never be entirely factual an...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [2]:
9  Broader Impact  This work offers several positive societal beneﬁts over previous work: the fact that it is more strongly grounded in real factual k...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf

Result [3]:
4.4 Fact Veriﬁcation  Table 2 shows our results on FEVER. For 3-way classiﬁcation, RAG scores are within 4.3% of state-of-the-art models, which are co...
Source: test_documents/rag-for-knowledge-intensive-nlp-tasks.pdf



# **Phase 5: Generation**

In [39]:
# LLM Integration - Initialize Gemini for answer generation

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.3,
    max_tokens=1024,
)

logger.info("Gemini LLM initialized")

2026-02-09 08:49:12 - google_genai._api_client - WARNING - Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.
2026-02-09 08:49:12 - IndustrialRAG - INFO - Gemini LLM initialized


In [40]:
# RAG Prompt - Define how to combine context with query

from langchain_core.prompts import ChatPromptTemplate

rag_prompt_template = """You are a helpful AI assistant. Answer the question based on the provided context.

Context:
{context}

Question: {question}

Instructions:
- Answer based ONLY on the information in the context
- If the context doesn't contain enough information, say "I don't have enough information to answer that."
- Be concise and accurate
- Use natural language

Answer:"""

rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

In [41]:
# Phase 5: RAG Chain - Combine retrieval, re-ranking, and generation

def generate_answer(query: str) -> str:
    """
    Complete RAG pipeline: retrieve -> re-rank -> generate answer.
    
    Args:
        query: User question
        
    Returns:
        Generated answer as string
    """
    # Step 1: Retrieve top 10 candidates
    candidates = retriever.invoke(query)
    logger.info(f"Retrieved {len(candidates)} candidates")
    
    # Step 2: Re-rank to top 3
    top_docs = rerank_documents(query, candidates, top_k=3)
    logger.info(f"Re-ranked to top {len(top_docs)} documents")
    
    # Step 3: Format context from top documents
    context = "\n\n".join([
        f"[Document {i+1}]\n{doc.page_content}" 
        for i, doc in enumerate(top_docs)
    ])
    
    # Step 4: Create prompt with context and question
    messages = rag_prompt.format_messages(context=context, question=query)
    
    # Step 5: Generate answer with LLM
    response = llm.invoke(messages)
    
    logger.info("Answer generated")
    
    return response.content

In [42]:
# Test complete RAG system

test_questions = [
    "What are the benefits of RAG systems?",
    "How does RAG work?",
    # "What are the challenges in RAG?"
]

for question in test_questions:
    print("="*80)
    print(f"Question: {question}")
    print("-"*80)
    
    answer = generate_answer(question)
    
    print(f"Answer:\n{answer}")
    print("="*80)
    print()

Question: What are the benefits of RAG systems?
--------------------------------------------------------------------------------


2026-02-09 08:51:34 - IndustrialRAG - INFO - Retrieved 10 candidates


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-09 08:51:34 - IndustrialRAG - INFO - Re-ranked 10 docs to top 3
2026-02-09 08:51:34 - IndustrialRAG - INFO - Re-ranked to top 3 documents
2026-02-09 08:51:34 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2026-02-09 08:51:37 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2026-02-09 08:51:37 - IndustrialRAG - INFO - Answer generated


Answer:
RAG systems offer several benefits: they are more strongly grounded in real factual knowledge, leading to less "hallucination" and more factual generations. They also provide more control and interpretability. RAG can be employed in various scenarios to benefit society, such as answering open-domain medical questions or helping people be more effective at their jobs. Additionally, RAG scores are competitive with state-of-the-art models without requiring complex pipeline systems, domain-specific architectures, substantial engineering, or intermediate retrieval supervision.

Question: How does RAG work?
--------------------------------------------------------------------------------


2026-02-09 08:51:38 - IndustrialRAG - INFO - Retrieved 10 candidates


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2026-02-09 08:51:38 - IndustrialRAG - INFO - Re-ranked 10 docs to top 3
2026-02-09 08:51:38 - IndustrialRAG - INFO - Re-ranked to top 3 documents
2026-02-09 08:51:38 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2026-02-09 08:51:40 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2026-02-09 08:51:40 - IndustrialRAG - INFO - Answer generated


Answer:
RAG works by combining information retrieval with language generation. The process involves several steps: documents are ingested and processed, then split into smaller chunks. Each chunk is converted into a dense vector representation using embedding models, and these embeddings are stored in a vector database. When a user asks a question, it is also converted into an embedding.

